In [ ]:





Search Dartmouth Formula Racing


1

2

1
3

4


Home
1

DMs
2

Activity
3

Later
4

More
0

Dartmouth Formula Racing














8990_traction




9




Messages

Add canvas

Files

Try a template NEW
CanvasListBookmark folder
Loading history...


Sasha
  8:41 PM
NON diverging IIR values:
static float IIR_raw_impulse[FILTER_LENGTH + 1] = {0.0048, 0.0193, 0.0289, 0.0193, 0.0048};
// a coefficients (denominator, excluding a[0] which is normalized to 1)
static float IIR_filtered_impulse[FILTER_LENGTH] = {-1.1958, 0.6900, -0.2341, 0.0388};
8:43
diverging IIR values:
Denominator (a):
    1.0000   -3.8358    5.5208   -3.5335    0.8486
Numerator (b):
   1.0e-05 *
    0.0898    0.3594    0.5391    0.3594    0.0898


will foox
  8:52 PM
https://docs.google.com/spreadsheets/d/10q3PUizuVPEDPAto-D3VKMm-kLkkxFO_qSJ9EmB44XE/edit?gid=0#gid=0


Liam Cotter
  8:52 PM
probably not worth anything now but I just realized I've been sending an older simulation method here's the actual one from before the midterm review
Simulation.ipynb
 
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
Click to expand inline (1,310 lines)




Liam Cotter
  9:05 PM
a  [ 1.         -3.67172909  5.06799839 -3.11596693  0.71991033]
b  [1.32937289e-05 5.31749156e-05 7.97623734e-05 5.31749156e-05
 1.32937289e-05]


Liam Cotter
  9:14 PM
a  [ 1.         -3.18063855  3.86119435 -2.11215536  0.43826514]
b  [0.0004166 0.0016664 0.0024996 0.0016664 0.0004166]


Wyatt Ellison
  11:37 PM
@Aaryan Agrawal
DataAnalysis.ipynb
 
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "42c106be-6cdc-4e66-84c5-6d8f415bc42e",
   "metadata": {},
   "source": [
    "todo:\n",
    "\n",
    "ID's for CAN Bus sensors not found in data. need help\n",
    "\n",
    "also data doesn't record multiple sensors at once (times are different)\n",
    "\n",
    "ADD PARAMETER FOR GRAPHS SO CAN CHOOSE START TIME AND END TIME\n",
    "\n",
    "find ids for \n",
    "/**\n",
    " * @brief Parse Sensor Board CAN messages.\n",
    " * @param frame: A CAN frame containing 8 bytes of data to be parsed.\n",
    " * @retval None\n",
    " */\n",
    "void CAN_Parse_SensorBoard() {\n",
    "    if (CAN_data_parsed[SENSORBOARD_Data]) {\n",
    "        return;\n",
    "    }\n",
    "    uint8_t data[8] = {0};\n",
    "    for (uint8_t i = 0; i < 8; i++) {\n",
    "        data[i] = (CAN_data[SENSORBOARD_Data] >> (i * 8)) & 0xFF;\n",
    "    }\n",
    "\n",
    "    // Byte 0, 1: Right Wheel RPM\n",
    "    CVC_data[SENSOR_RIGHT_WHEELSPEED] = data[0] | (data[1] << 8);\n",
    "    // Byte 2, 3: Left Wheel RPM\n",
    "    CVC_data[SENSOR_LEFT_WHEELSPEED] = data[2] | (data[3] << 8);\n",
    "    // Byte 4: Brake Pressure\n",
    "    CVC_data[SENSOR_BRAKESWITCH] = data[4];\n",
    "    // Byte 5: Steering Angle\n",
    "    CVC_data[SENSOR_STEERING_ANGLE] = data[5];\n",
    "    // Byte 6, 7: Throttle Position\n",
    "    CVC_data[SENSOR_THROTTLE_ADC] = data[6] | (data[7] << 8);\n",
    "\n",
    "    CAN_data_parsed[SENSORBOARD_Data] = true;\n",
    "}"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "e9d6a446-0e88-420d-a02a-c42b8e428943",
   "metadata": {},
   "source": [
    "To Run:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "ca71dc19",
   "metadata": {},
   "outputs": [],
   "source": [
    "%matplotlib inline "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "644e7154-c45f-46f2-8fea-21f0117477fe",
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import os\n",
    "from sklearn.decomposition import PCA\n",
    "from sklearn.preprocessing import StandardScaler"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "id": "82a6dff4-1b78-4b18-8881-f5e12ad5564a",
   "metadata": {},
   "outputs": [],
   "source": [
    "# to get directory, drag example file into terminal\n",
    "\n",
    "# directory = '/Users/liam/Desktop/Dartmouth/Engs 89/Data/decoded'\n",
    "directory = '/Users/wellison/Documents/Dartmouth/24F/ES89_90/Data-analysis/'\n",
    "\n",
    "CAN_SENSORBOARD_BASE_11 = 0x65D\n",
    "\n",
    "CAN_SENSORBOARD_IDS = {\n",
    "    \"Right_Wheel_RPM\": CAN_SENSORBOARD_BASE_11 + 0x01,\n",
    "    \"Left_Wheel_RPM\": CAN_SENSORBOARD_BASE_11 + 0x02,\n",
    "    \"Brake_Pressure\": CAN_SENSORBOARD_BASE_11 + 0x03,\n",
    "    \"Steering_Angle\": CAN_SENSORBOARD_BASE_11 + 0x04,\n",
    "    \"Throttle_Position\": CAN_SENSORBOARD_BASE_11 + 0x05\n",
    "}"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "id": "974aefe4",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "/Users/wellison/Documents/Dartmouth/24F/ES89_90/Data-analysis/.DS_Store\n",
      "/Users/wellison/Documents/Dartmouth/24F/ES89_90/Data-analysis/DCB\n",
      "/Users/wellison/Documents/Dartmouth/24F/ES89_90/Data-analysis/drive-data\n",
      "/Users/wellison/Documents/Dartmouth/24F/ES89_90/Data-analysis/method_accuracy.mlx\n",
      "/Users/wellison/Documents/Dartmouth/24F/ES89_90/Data-analysis/baselineData.zip\n",
      "/Users/wellison/Documents/Dartmouth/24F/ES89_90/Data-analysis/real-time-filtering-simulator.ipynb\n",
      "/Users/wellison/Documents/Dartmouth/24F/ES89_90/Data-analysis/RPM_validation.mlx\n",
      "/Users/wellison/Documents/Dartmouth/24F/ES89_90/Data-analysis/HARDWARE_QUADRATURE_CAN.TXT\n",
      "/Users/wellison/Documents/Dartmouth/24F/ES89_90/Data-analysis/.ipynb_checkpoints\n",
      "/Users/wellison/Documents/Dartmouth/24F/ES89_90/Data-analysis/method_accuracy.m\n",
      "/Users/wellison/Documents/Dartmouth/24F/ES89_90/Data-analysis/baselineData\n",
      "/Users/wellison/Documents/Dartmouth/24F/ES89_90/Data-analysis/baselineFrontData_01_30_25\n",
      "/Users/wellison/Documents/Dartmouth/24F/ES89_90/Data-analysis/data_filtering.mlx\n",
      "/Users/wellison/Documents/Dartmouth/24F/ES89_90/Data-analysis/DataAnalysis.ipynb\n"
     ]
    }
   ],
   "source": [
    "for filename in os.listdir(directory):\n",
    "\n",
    "    file_path = os.path.join(directory, filename)\n",
    "    print(file_path)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "id": "9eedc15d-f65b-4434-b278-3212fde66ad8",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Warning: File HARDWARE_QUADRATURE_CAN.TXT is missing 'Time' or 'Speed' columns. Skipping.\n"
     ]
    },
    {
     "ename": "KeyError",
     "evalue": "\"None of [Index(['Time'], dtype='object')] are in the [columns]\"",
     "output_type": "error",
     "traceback": [
      "\u001b[0;31m---------------------------------------------------------------------------\u001b[0m",
      "\u001b[0;31mKeyError\u001b[0m                                  Traceback (most recent call last)",
      "\u001b[0;32m<ipython-input-15-cfbcf3f024ae>\u001b[0m in \u001b[0;36m<module>\u001b[0;34m\u001b[0m\n\u001b[1;32m     46\u001b[0m \u001b[0;31m# Select only the 'Time' column and columns ending with 'Speed'\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     47\u001b[0m \u001b[0mcols_to_keep\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0;34m[\u001b[0m\u001b[0;34m'Time'\u001b[0m\u001b[0;34m]\u001b[0m \u001b[0;34m+\u001b[0m \u001b[0;34m[\u001b[0m\u001b[0mcol\u001b[0m \u001b[0;32mfor\u001b[0m \u001b[0mcol\u001b[0m \u001b[0;32min\u001b[0m \u001b[0mdf\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mcolumns\u001b[0m \u001b[0;32mif\u001b[0m \u001b[0mcol\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mendswith\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m'Speed'\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m---> 48\u001b[0;31m \u001b[0mdf\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mdf\u001b[0m\u001b[0;34m[\u001b[0m\u001b[0mcols_to_keep\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m     49\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m     50\u001b[0m \u001b[0;31m# If the DataFrame is valid, append it to the list\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m~/opt/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py\u001b[0m in \u001b[0;36m__getitem__\u001b[0;34m(self, key)\u001b[0m\n\u001b[1;32m   3765\u001b[0m             \u001b[0;32mif\u001b[0m \u001b[0mis_iterator\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mkey\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   3766\u001b[0m                 \u001b[0mkey\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mlist\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mkey\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 3767\u001b[0;31m             \u001b[0mindexer\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mcolumns\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_get_indexer_strict\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mkey\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0;34m\"columns\"\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m[\u001b[0m\u001b[0;36m1\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m   3768\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   3769\u001b[0m         \u001b[0;31m# take() does not accept boolean indexers\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m~/opt/miniconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py\u001b[0m in \u001b[0;36m_get_indexer_strict\u001b[0;34m(self, key, axis_name)\u001b[0m\n\u001b[1;32m   5875\u001b[0m             \u001b[0mkeyarr\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mindexer\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mnew_indexer\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_reindex_non_unique\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mkeyarr\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   5876\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 5877\u001b[0;31m         \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0m_raise_if_missing\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mkeyarr\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0mindexer\u001b[0m\u001b[0;34m,\u001b[0m \u001b[0maxis_name\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m   5878\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   5879\u001b[0m         \u001b[0mkeyarr\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mself\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mtake\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mindexer\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;32m~/opt/miniconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py\u001b[0m in \u001b[0;36m_raise_if_missing\u001b[0;34m(self, key, indexer, axis_name)\u001b[0m\n\u001b[1;32m   5936\u001b[0m                 \u001b[0;32mif\u001b[0m \u001b[0muse_interval_msg\u001b[0m\u001b[0;34m:\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   5937\u001b[0m                     \u001b[0mkey\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mlist\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mkey\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0;32m-> 5938\u001b[0;31m                 \u001b[0;32mraise\u001b[0m \u001b[0mKeyError\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34mf\"None of [{key}] are in the [{axis_name}]\"\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n\u001b[0m\u001b[1;32m   5939\u001b[0m \u001b[0;34m\u001b[0m\u001b[0m\n\u001b[1;32m   5940\u001b[0m             \u001b[0mnot_found\u001b[0m \u001b[0;34m=\u001b[0m \u001b[0mlist\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mensure_index\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0mkey\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m[\u001b[0m\u001b[0mmissing_mask\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0mnonzero\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m[\u001b[0m\u001b[0;36m0\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m]\u001b[0m\u001b[0;34m.\u001b[0m\u001b[0munique\u001b[0m\u001b[0;34m(\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m)\u001b[0m\u001b[0;34m\u001b[0m\u001b[0;34m\u001b[0m\u001b[0m\n",
      "\u001b[0;31mKeyError\u001b[0m: \"None of [Index(['Time'], dtype='object')] are in the [columns]\""
     ]
    }
   ],
   "...
Collapse
 This snippet was truncated for display; see it in full







NEW



Message 8990_traction









Shift + Enter to add a new line



